In [1]:
#!pip install seaborn 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import tqdm as tqdm
sns.__version__

'0.11.2'

In [2]:
FOLDER = 'data'
FILE = '20-years-odds.csv'


# modeling & feature engineering constant
# number of game to look before to create feature
NB_PREV_GAME = 3

data = pd.read_csv(f'{FOLDER}/{FILE}')
# rate for the test set

data.shape

(65960, 17)

In [3]:
#data = data.query('competition.str.contains("premier-league") & period > "2011"', engine='python').copy()
LIMIT = int(data.shape[0]*0.85)
data


,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,serie-a-2021,2.0,2.0,N,1.0,4.03,False
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,3.0,2,2.0,1.88,True
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,serie-a-2021,4.0,0.0,1,0.0,1.29,True
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,2.0,2,2.0,1.52,True


In [4]:
teams = [data.home_team.tolist()] + [data.away_team.tolist()]

teams = np.unique(teams[0])
len(np.unique(teams))

288

In [5]:
scores = np.unique(data.correct_score)
df_scores = pd.DataFrame(data.correct_score.value_counts(normalize=True))
#df_scores.columns = [['scores','ratio']]
#df_scores.sort_values(ascending=False)
df_scores.columns = ['ratio']
df_scores.sort_values('ratio',ascending=False)

,ratio
1:1,0.129518
1:0,0.108475
2:1,0.091343
0:0,0.084991
2:0,0.080033
...,...
9:1,0.000015
6:5,0.000015
4:7,0.000015
1:9,0.000015


# Baseline

Sinmply compute a baseline by taking the 1N2 odds, transpose them to 1/odds then scale date and train a multi-output regressor.

## Result

Depending on the choice of the estimator the score (accuracy) is between 41-48 % of good prediction about game outcome.
For the correct score, the prediction is around 11% of good result.

In [6]:
df = data[['1N2_1','1N2_N','1N2_2','home_score','away_score']]
df

,1N2_1,1N2_N,1N2_2,home_score,away_score
0,1.30,4.70,8.00,1.0,0.0
1,1.65,3.60,4.00,0.0,1.0
2,1.50,3.50,6.00,2.0,1.0
3,3.00,2.90,2.40,2.0,2.0
4,1.55,3.30,5.80,3.0,0.0
...,...,...,...,...,...
65955,6.85,4.03,1.53,2.0,2.0
65956,4.04,3.75,1.88,1.0,3.0
65957,1.29,5.53,10.81,4.0,0.0
65958,6.12,4.46,1.52,1.0,2.0


In [7]:
X = 1/df[['1N2_1','1N2_N','1N2_2']]
y = df.drop(['1N2_1','1N2_N','1N2_2'],axis=1)
# summarize dataset
print(X.shape, y.shape)

(65960, 3) (65960, 2)


In [8]:
X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(56066, 3) (9894, 3) (56066, 2) (9894, 2)


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
model = LinearRegression(fit_intercept=False)



model = DecisionTreeRegressor()
model = KNeighborsRegressor(n_neighbors=11)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=11)

In [11]:

yhat = model.predict(X_test)
eval = y_test.copy()

yhat49=yhat+0.4999
yhat49=yhat49.astype(int)

eval['home_pred']=yhat49[:,0]
eval['away_pred']=yhat49[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score'])),
    ((eval['home_pred'] < eval['away_pred']) & (eval['home_score'] < eval['away_score']))
    
]
choices = [True,True,True]

eval['prediction_1N2'] = np.select(conditions, choices, default=False)


conditions = [
    ((eval['home_pred'] == eval['home_score']) & (eval['away_pred'] == eval['away_score']))
]
choices = [True]

eval['prediction_score'] = np.select(conditions, choices, default=False)


eval.prediction_1N2.value_counts(normalize=True),eval.prediction_score.value_counts(normalize=True)


(False    0.560441
 True     0.439559
 Name: prediction_1N2, dtype: float64,
 False    0.889125
 True     0.110875
 Name: prediction_score, dtype: float64)

In [12]:
eval

,home_score,away_score,home_pred,away_pred,prediction_1N2,prediction_score
56066,2.0,1.0,2,0,True,False
56067,1.0,2.0,2,1,False,False
56068,1.0,1.0,1,2,False,False
56069,1.0,1.0,2,1,False,False
56070,2.0,0.0,2,0,True,True
...,...,...,...,...,...,...
65955,2.0,2.0,1,2,False,False
65956,1.0,3.0,1,2,True,False
65957,4.0,0.0,2,1,True,False
65958,1.0,2.0,1,1,False,False


# Features Engineering

This section will add some features to the base line to see if we can improve it

In [13]:
data

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,serie-a-2021,2.0,2.0,N,1.0,4.03,False
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,3.0,2,2.0,1.88,True
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,serie-a-2021,4.0,0.0,1,0.0,1.29,True
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,2.0,2,2.0,1.52,True


In [14]:
data.query('home_team == "Arsenal" & away_team == "Aston Villa"')[['home_score','away_score','odd_win']]

,home_score,away_score,odd_win
0,1.0,0.0,1.30
2718,3.0,1.0,1.63
5134,1.0,0.0,1.58
7322,3.0,2.0,1.61
12137,2.0,0.0,1.32
15045,3.0,1.0,1.28
16970,5.0,0.0,1.28
20946,1.0,1.0,3.51
24228,0.0,2.0,6.90
26265,3.0,0.0,1.68


In [15]:
import time as time
B = []
A = time.time()


"""
For each game the method must collect for the home team the last 3 games
- it can be home or away game

Then:
- make the sum() of goals scored
- make the sum() of goals away

Store:
home_3_scored
home_3_conceded

away_3_scored
away_3_conceded

Method
1. take home team and look last 3 game, team can be in home or away
2. depending on home or away make total
3. store data in home_3_xxxx
4. do 1,2 and 3 for away team

"""

for idx, r in (data.iterrows()):
    t1 = r.home_team

    dt = r.matchdate
    
    # home team
    ht = data.query('(home_team == @t1 | away_team == @t1) & matchdate < @dt').sort_values('matchdate', ascending=False).head(3)
    
    b1 = ht.query('home_team == @t1')['home_score'].sum()
    a1 = ht.query('home_team == @t1')['away_score'].sum()
    
    b2 = ht.query('away_team == @t1')['away_score'].sum()
    a2 = ht.query('away_team == @t1')['home_score'].sum()
    
    data.loc[idx,'home_3_scored']=b1+b2
    data.loc[idx,'home_3_conceded']=a1+a2
    
    # away_team
    t1 = r.away_team
    
    ht = data.query('(home_team == @t1 | away_team == @t1) & matchdate < @dt').sort_values('matchdate', ascending=False).head(3)
    
    b1 = ht.query('home_team == @t1')['home_score'].sum()
    a1 = ht.query('home_team == @t1')['away_score'].sum()
    
    b2 = ht.query('away_team == @t1')['away_score'].sum()
    a2 = ht.query('away_team == @t1')['home_score'].sum()
    
    data.loc[idx,'away_3_scored']=b1+b2
    data.loc[idx,'away_3_conceded']=a1+a2

B.append(round(time.time()-A,5))    



data

    
    

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,...,home_score,away_score,bet_won,target,odd_win,best_odd_won,home_3_scored,home_3_conceded,away_3_scored,away_3_conceded
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,...,1.0,0.0,1,0.0,1.30,True,4.0,2.0,6.0,6.0
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,...,0.0,1.0,2,2.0,4.00,False,5.0,7.0,1.0,2.0
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,...,2.0,1.0,1,0.0,1.50,True,6.0,3.0,2.0,2.0
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,...,2.0,2.0,N,1.0,2.90,False,2.0,2.0,6.0,2.0
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,...,3.0,0.0,1,0.0,1.55,True,5.0,5.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,...,2.0,2.0,N,1.0,4.03,False,2.0,11.0,10.0,4.0
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,...,1.0,3.0,2,2.0,1.88,True,6.0,3.0,0.0,4.0
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,...,4.0,0.0,1,0.0,1.29,True,10.0,5.0,6.0,4.0
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,...,1.0,2.0,2,2.0,1.52,True,2.0,5.0,6.0,5.0


In [22]:
data.to_csv('data/dataset_engineering-01.csv',index=False)

In [16]:
data.query('home_team == "Arsenal" | away_team == "Arsenal"').sort_values('matchdate', ascending=False)[['matchdate','home_team','away_team','1N2_1','1N2_N','1N2_2','odd_win','home_3_scored','home_3_conceded','away_3_scored','away_3_conceded','home_score','away_score']]

,matchdate,home_team,away_team,1N2_1,1N2_N,1N2_2,odd_win,home_3_scored,home_3_conceded,away_3_scored,away_3_conceded,home_score,away_score
62650,2022-05-22,Arsenal,Everton,1.34,5.50,8.98,1.34,2.0,6.0,5.0,5.0,5.0,1.0
62664,2022-05-16,Newcastle,Arsenal,4.28,3.78,1.84,4.28,3.0,6.0,4.0,5.0,2.0,0.0
62672,2022-05-12,Tottenham,Arsenal,2.28,3.45,3.21,2.28,4.0,2.0,7.0,3.0,3.0,0.0
62678,2022-05-08,Arsenal,Leeds,1.52,4.51,6.18,1.52,9.0,4.0,3.0,4.0,2.0,1.0
62691,2022-05-01,West Ham,Arsenal,4.43,3.92,1.79,1.79,1.0,4.0,7.0,4.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,1998-09-12,Leicester,Arsenal,3.10,2.95,2.25,2.95,2.0,2.0,0.0,0.0,1.0,1.0
342,1998-09-09,Chelsea,Arsenal,2.45,2.85,2.95,2.85,2.0,3.0,2.0,1.0,0.0,0.0
352,1998-08-29,Arsenal,Charlton,1.30,4.50,8.00,4.50,2.0,1.0,5.0,0.0,0.0,0.0
366,1998-08-22,Liverpool,Arsenal,2.40,2.80,3.10,2.80,2.0,1.0,2.0,1.0,0.0,0.0


In [17]:
data.query('home_team == "Everton" | away_team == "Everton"').sort_values('matchdate', ascending=False)[['matchdate','home_team','away_team','1N2_1','1N2_N','1N2_2','odd_win','home_3_scored','home_3_conceded','away_3_scored','away_3_conceded','home_score','away_score']]

,matchdate,home_team,away_team,1N2_1,1N2_N,1N2_2,odd_win,home_3_scored,home_3_conceded,away_3_scored,away_3_conceded,home_score,away_score
62650,2022-05-22,Arsenal,Everton,1.34,5.50,8.98,1.34,2.0,6.0,5.0,5.0,5.0,1.0
62660,2022-05-19,Everton,Crystal Palace,1.86,3.50,4.58,1.86,4.0,4.0,4.0,2.0,3.0,2.0
62666,2022-05-15,Everton,Brentford,1.90,3.60,4.22,4.22,3.0,1.0,3.0,3.0,2.0,3.0
62675,2022-05-11,Watford,Everton,5.71,3.77,1.66,3.77,2.0,8.0,3.0,3.0,0.0,0.0
62679,2022-05-08,Leicester,Everton,2.65,3.25,2.80,2.80,2.0,4.0,2.0,3.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,1998-09-12,Everton,Leeds,2.85,2.95,2.85,2.95,2.0,3.0,5.0,1.0,0.0,0.0
350,1998-09-08,Nottingham,Everton,1.90,3.15,3.80,3.80,4.0,3.0,0.0,3.0,0.0,2.0
355,1998-08-29,Everton,Tottenham,2.10,3.00,3.40,3.40,0.0,2.0,1.0,6.0,0.0,1.0
365,1998-08-22,Leicester,Everton,1.80,3.00,4.30,1.80,2.0,2.0,0.0,0.0,2.0,0.0


In [18]:
df = data[['1N2_1','1N2_N','1N2_2','home_3_scored','home_3_conceded','away_3_scored','away_3_conceded','home_score','away_score']].fillna(0)
df

,1N2_1,1N2_N,1N2_2,home_3_scored,home_3_conceded,away_3_scored,away_3_conceded,home_score,away_score
0,1.30,4.70,8.00,4.0,2.0,6.0,6.0,1.0,0.0
1,1.65,3.60,4.00,5.0,7.0,1.0,2.0,0.0,1.0
2,1.50,3.50,6.00,6.0,3.0,2.0,2.0,2.0,1.0
3,3.00,2.90,2.40,2.0,2.0,6.0,2.0,2.0,2.0
4,1.55,3.30,5.80,5.0,5.0,2.0,5.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...
65955,6.85,4.03,1.53,2.0,11.0,10.0,4.0,2.0,2.0
65956,4.04,3.75,1.88,6.0,3.0,0.0,4.0,1.0,3.0
65957,1.29,5.53,10.81,10.0,5.0,6.0,4.0,4.0,0.0
65958,6.12,4.46,1.52,2.0,5.0,6.0,5.0,1.0,2.0


In [19]:

model_cols = ['1N2_1','1N2_N','1N2_2','home_3_scored','home_3_conceded','away_3_scored','away_3_conceded']
X = df[model_cols]
y = df.drop(model_cols,axis=1)
# summarize dataset
print(X.shape, y.shape)

X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(65960, 7) (65960, 2)
(56066, 7) (9894, 7) (56066, 2) (9894, 2)


In [20]:
model = LinearRegression(fit_intercept=False)



model = DecisionTreeRegressor()
model = KNeighborsRegressor(n_neighbors=11)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=11)

In [21]:
yhat = model.predict(X_test)
eval = y_test.copy()

yhat49=yhat+0.4999
yhat49=yhat49.astype(int)

eval['home_pred']=yhat49[:,0]
eval['away_pred']=yhat49[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score'])),
    ((eval['home_pred'] < eval['away_pred']) & (eval['home_score'] < eval['away_score']))
    
]
choices = [True,True,True]

eval['prediction_1N2'] = np.select(conditions, choices, default=False)


conditions = [
    ((eval['home_pred'] == eval['home_score']) & (eval['away_pred'] == eval['away_score']))
]
choices = [True]

eval['prediction_score'] = np.select(conditions, choices, default=False)


eval.prediction_1N2.value_counts(normalize=True),eval.prediction_score.value_counts(normalize=True)

(False    0.567212
 True     0.432788
 Name: prediction_1N2, dtype: float64,
 False    0.882151
 True     0.117849
 Name: prediction_score, dtype: float64)